In [7]:
print(torch.backends.mps.is_available())

True


In [15]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")

llm.invoke("Tell me a joke")

" Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [11]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "mistralai/Mixtral-8x7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map='mps')

# text = "Hello my name is"
# inputs = tokenizer(text, return_tensors="pt").to("mps")

# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))